# 애쓰는 감자 최종 코드

### 환경
- Ubuntu 20.04, GPU = RTX3090 x 2
- Colab pro

In [1]:
### local 기준 작성

## import module

In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/gdrive/


In [ ]:
import os
colab_path = '/content/drive/KED/'
os.chdir(colab_path) # colab 작업환경으로 디렉토리 변경

## Base PreProcessing

In [ ]:
train_txt_path = 'train.txt'
data = pd.read_csv(train_txt_path,sep="\", engine='python', encoding="cp949", dtype = {'KEDCD':str})

In [ ]:
kedcd = data.KEDCD.unique()
ksic = data.KSIC10_BZC_CD.unique()
data2 = data[data.notnull()]
# 숫자 제거
data2['BZ_PPOS_ITM_CTT'] = data2['BZ_PPOS_ITM_CTT'].str.replace("^[0-9]",'')
data2.head(10)

,KEDCD,SEQ,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD
0,0007709505,1,"건축자재 도, 소매업.",G46699
1,0007709505,2,부동산업.,G46699
2,0007709505,3,주택사업.,G46699
3,0007709505,4,각 호에관련된 부대사업 일체.,G46699
4,0007866121,1,"농산물의 제조, 유통, 가공, 판매",C10309
5,0007866121,2,농작업의 전부 또는 일부 대행,C10309
6,0007866121,3,농산물 연구사업,C10309
7,0007866121,4,인터넷상품중개업,C10309
8,0007866121,5,전자상거래업,C10309
9,0007866121,6,과실 및 채소 저장처리업,C10309


In [ ]:
dic_x = data2.groupby(['KEDCD','KSIC10_BZC_CD'])['BZ_PPOS_ITM_CTT'].apply(lambda x: x.values.tolist()).to_dict()

In [ ]:
df = pd.DataFrame(columns=['KEDCD','BZ_PPOS_ITM_CTT','KSIC10_BZC_CD'])
df['KEDCD'] = list(map(lambda x: x[0],list(dic_x.keys())))
df['KSIC10_BZC_CD'] = list(map(lambda x: x[1],list(dic_x.keys())))
df['BZ_PPOS_ITM_CTT'] = dic_x.values()

In [ ]:
df['BZ_PPOS_ITM_CTT'] = df['BZ_PPOS_ITM_CTT'].apply(lambda x: ''.join(x))

In [ ]:
df.isnull().sum()

KEDCD              0
BZ_PPOS_ITM_CTT    0
KSIC10_BZC_CD      0
dtype: int64

In [ ]:
df['Big'] = df['KSIC10_BZC_CD'].apply(lambda x: x[0])

In [ ]:
from sklearn.model_selection import train_test_split
train,val = train_test_split(df,test_size = .3, random_state = 42)

In [ ]:
test = pd.read_csv('test.txt',dtype={'KEDCD':str})
test.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
kedcd = test.KEDCD.unique()
test = test[test.notnull()]
test['BZ_PPOS_ITM_CTT'] = test['BZ_PPOS_ITM_CTT'].str.replace('[^\w\s]', '', regex=True)
test['BZ_PPOS_ITM_CTT'] = test['BZ_PPOS_ITM_CTT'].str.replace("^[0-9]",'')


In [ ]:
dic = test.groupby(['KEDCD'])['BZ_PPOS_ITM_CTT'].apply(lambda x: x.values.tolist()).to_dict()
test_df = pd.DataFrame(columns=['KEDCD','BZ_PPOS_ITM_CTT'])
test_df['KEDCD'] = dic.keys()
test_df['BZ_PPOS_ITM_CTT'] = dic.values()
test_df['BZ_PPOS_ITM_CTT'] = test_df['BZ_PPOS_ITM_CTT'].apply(lambda ls: [x for x in ls if str(x) != 'nan'])
test_df['BZ_PPOS_ITM_CTT'] = test_df['BZ_PPOS_ITM_CTT'].apply(lambda x: ''.join(x))

In [ ]:
# test_df.to_csv('test.csv',index=False)

,KEDCD,BZ_PPOS_ITM_CTT
0,1000313,주택건설사업 대지조성업 임대주택사업 토목건축 공사업 도로포장 공사업 전기공사 청부...
1,1000929,섬유 제조업 섬유 무역업 섬유 수출입업 부동산 임대업 위 각호에 관련된 부대사업 일체
2,1002091,토목공사업 건축공사업 상하수도 공사업 문화재 보수 공사업 부동산 개발업 주택공사업...
3,10116,부동산 건설업 부동산 개발업 부동산 금융업 부동산 매매 및 임대업 부동산 이용 개...
4,101397,사업 협업적 농업경영을 통하여 생산성을 높이고 조합원의 소득증대를 도모함을 목적으...
...,...,...
19154,991555,농약신약제약개발 유용생물소재의 산업화 병저항성관련 유전자의 산업화 유전적지법에 의...
19155,995892,전자부품제품 제조 및 판매업 가전기기 제조 및 판매업 주택설비기기 제조 및 판매업...
19156,998792,영농경영의 합리화로 농업생산성의 향상과 조합원의 소득증대를 도모함을 목적으로하고 그...
19157,998996,의료기기관련 소프트웨어 개발 및 판매 멀티미디어 개발 및 판매 의료기기관련 하드웨...


In [ ]:
# train.to_csv('/content/drive/MyDrive/애쓰는 감자/data/train.csv',index=False)
# val.to_csv('/content/drive/MyDrive/애쓰는 감자/data/train.csv',index=False)

## Augmentation Data

In [ ]:
data = pd.read_csv('rename.csv') # 한국표준산업분류(10차)_표.xlsx를 python에서 불러올 수 있는 형태로 변환된 csv

In [ ]:
data.drop([0,1],axis=0,inplace=True)
data.columns = [str(i) for i in range(len(data.columns))]

In [ ]:
df = pd.DataFrame(columns=['Dis','Target'])

In [ ]:
df['Target'] = data['0']
df['Dis'] = data['1']
df = df.dropna().reset_index(drop=True)

In [ ]:
# 증강 데이터 정리
## DIS = 한국어 설명 ## Target = 예측할 코드 번호
tmp = data[['3','2']]
tmp.columns = ['Dis','Target']
df = pd.concat([df,tmp]).dropna()
tmp2 = data[['5','4']]
tmp2.columns = ['Dis','Target']
df = pd.concat([df,tmp2]).dropna()
tmp3 = data[['7','6']]
tmp3.columns = ['Dis','Target']
df = pd.concat([df,tmp3]).dropna()
tmp4 = data[['9','8']]
tmp4.columns = ['Dis','Target']
df = pd.concat([df,tmp4]).dropna()

In [ ]:
# df.to_csv('Augmentation_data.csv',index=False)

## PreProcessing for Bert

### 대분류

In [ ]:
train = pd.read_csv('train.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str})
plus = pd.read_csv('Augmentation_data.csv')

In [ ]:
import re
plus["BZ_PPOS_ITM_CTT"] = plus["Dis"].str.replace('[^\w\s]', '', regex=True) # 특수문자 제거
plus.head()

In [ ]:
plus["BZ_PPOS_ITM_CTT"] = plus["BZ_PPOS_ITM_CTT"].str.replace('[0-9]+', '', regex=True) # 숫자 제거
plus.head()

In [ ]:
plus["Big"] = plus["Target"]
plus.head()

In [ ]:
data_big = plus.dropna(axis=0)
len(data_big)

In [ ]:
plus = plus.loc[:,['BZ_PPOS_ITM_CTT', 'Big']]
plus.head()

In [ ]:
plus['Big'] = plus['Big'].str[0]
plus

In [ ]:
trn = pd.concat([train, plus], ignore_index = True)
trn.tail()

In [ ]:
trn = trn.loc[:,['BZ_PPOS_ITM_CTT', 'Big']]
trn.head()

In [ ]:
trn =trn.dropna(axis=0)
len(trn)

In [ ]:
trn_shuffle[trn_shuffle['Big'] == "3"]

In [ ]:
remove_ = trn_shuffle[trn_shuffle['Big'] == "3"].index
trn_shuffle = trn_shuffle.drop(remove_)
len(trn_shuffle)

In [ ]:
trn_shuffle = trn.sample(frac=1).reset_index(drop=True)
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('train_data_se.csv', index = False)

In [ ]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(trn_shuffle['Big'])
trn_shuffle['Big_idx'] = encoder.transform(trn_shuffle['Big'])
trn_shuffle.head()

In [ ]:
# 라벨링된 것 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
trn_shuffle = trn_shuffle.loc[:,['BZ_PPOS_ITM_CTT', 'Big_idx']]
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('train_big.txt', sep='\t', index=True)

### 중분류

In [4]:
train = pd.read_csv('train.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str})
plus = pd.read_csv('Augmentation_data.csv')

In [5]:
import re
plus["BZ_PPOS_ITM_CTT"] = plus["Dis"].str.replace('[^\w\s]', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업0103
1,광업(05~08),B,광업0508
2,제조업(10~34),C,제조업1034
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업35
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업3639


In [6]:
plus["BZ_PPOS_ITM_CTT"] = plus["BZ_PPOS_ITM_CTT"].str.replace('[0-9]+', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업
1,광업(05~08),B,광업
2,제조업(10~34),C,제조업
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업


In [7]:
plus["mid"] = plus["Target"].str[1:3]
plus

,Dis,Target,BZ_PPOS_ITM_CTT,mid
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업,
1,광업(05~08),B,광업,
2,제조업(10~34),C,제조업,
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업,
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업,
...,...,...,...,...
2016,가구 내 고용활동,T97000,가구 내 고용활동,97
2017,자가 소비를 위한 가사 생산 활동,T98100,자가 소비를 위한 가사 생산 활동,98
2018,자가 소비를 위한 가사 서비스 활동,T98200,자가 소비를 위한 가사 서비스 활동,98
2019,주한 외국 공관,U99001,주한 외국 공관,99


In [8]:
# 추가내용 파일 - 해당 분류에 맞게 삭제 
for i in range(len(plus)):
    if len(plus['mid'][i])<2:
        plus=plus.drop(index=i)

print(len(plus))

1999


In [9]:
plus['mid'].unique()

array(['01', '02', '03', '05', '06', '07', '08', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '36',
       '37', '38', '39', '41', '42', '45', '46', '47', '49', '50', '51',
       '52', '55', '56', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '68', '70', '71', '72', '73', '74', '75', '76', '84', '85',
       '86', '87', '90', '91', '94', '95', '96', '97', '98', '99', '35'],
      dtype=object)

In [10]:
data_mid = plus.loc[:,['BZ_PPOS_ITM_CTT', 'mid']]
data_mid.tail()

,BZ_PPOS_ITM_CTT,mid
2016,가구 내 고용활동,97
2017,자가 소비를 위한 가사 생산 활동,98
2018,자가 소비를 위한 가사 서비스 활동,98
2019,주한 외국 공관,99
2020,기타 국제 및 외국기관,99


In [11]:
for i in range(len(train)):
    if len(train['KSIC10_BZC_CD'][i])<3:
        train=train.drop(index=i)

print(len(train))

1139566


In [12]:
train['mid']=train['KSIC10_BZC_CD'].str[1:3]
train.head()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big
0,7695516.0,농산물 판매업 전자상거래업 통신판매업농임산물 건강기능식품 건강보조식품 일반식품...,G46209,46,G
1,8177802.0,건축공사업 토목공사업 토공사업 토목건축공사업 철근콘크리트공사업 실내인테리...,F41229,41,F
2,8233352.0,음식점업 경양식 퓨전 한식 중식 일식 식품 제조 가공 유통 판매 도소매업 제과...,I56111,56,I
3,991833.0,변호사의 직무에 속하는 업무 공증인의 직무에 속하는 업무 변호사법 제조에 속하...,M71101,71,M
4,7305137.0,일반폐기물처리업 재활용사업 정화조 청소업 위 각항에 부대되는 사업일체,E38210,38,E


In [13]:
trn = pd.concat([train, data_mid], ignore_index = True)
trn.tail()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big
1141560,NaN,가구 내 고용활동,NaN,97,NaN
1141561,NaN,자가 소비를 위한 가사 생산 활동,NaN,98,NaN
1141562,NaN,자가 소비를 위한 가사 서비스 활동,NaN,98,NaN
1141563,NaN,주한 외국 공관,NaN,99,NaN
1141564,NaN,기타 국제 및 외국기관,NaN,99,NaN


In [14]:
trn = trn.loc[:,['BZ_PPOS_ITM_CTT', 'mid']]
trn.tail()

,BZ_PPOS_ITM_CTT,mid
1141560,가구 내 고용활동,97
1141561,자가 소비를 위한 가사 생산 활동,98
1141562,자가 소비를 위한 가사 서비스 활동,98
1141563,주한 외국 공관,99
1141564,기타 국제 및 외국기관,99


In [15]:
trn =trn.dropna(axis=0)
len(trn)

1141565

In [16]:
trn_shuffle = trn.sample(frac=1).reset_index(drop=True)
trn_shuffle.tail()

,BZ_PPOS_ITM_CTT,mid
1141560,합성수지 제조업 자동차부품 제조업 ...,20
1141561,생산하도급용역업 자동차부품조립가공검사포장제조업 인력관리 및 용역업 전자부품조립가공검...,75
1141562,는 협동적 자치정신을 현현하여 임업의 건전한 발전을 기함을 목적으로 한다 전조의 목...,02
1141563,농업의 경영 및 부대사업 집단재배 및 공동작업에 관한 사업 농업에 관련된 공동...,01
1141564,일반교구개발업일반교구제작업교육용 컴퓨터 프로그램개발업일반교구판매업교구판매대행 로봇...,27


In [18]:
trn_shuffle.to_csv('preprocessing_data/train_data_mid.csv', index = False)

In [ ]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(trn_shuffle['mid'])
trn_shuffle['mid_idx'] = encoder.transform(trn_shuffle['mid'])
trn_shuffle.head()

In [ ]:
# 라벨링된 것 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
trn_shuffle = trn_shuffle.loc[:,['BZ_PPOS_ITM_CTT', 'mid_idx']]
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('train_mid.txt', sep='\t', index=True)

### 소분류

In [28]:
train = pd.read_csv('train.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str})
plus = pd.read_csv('Augmentation_data.csv')

In [29]:
import re
plus["BZ_PPOS_ITM_CTT"] = plus["Dis"].str.replace('[^\w\s]', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업0103
1,광업(05~08),B,광업0508
2,제조업(10~34),C,제조업1034
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업35
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업3639


In [30]:
plus["BZ_PPOS_ITM_CTT"] = plus["BZ_PPOS_ITM_CTT"].str.replace('[0-9]+', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업
1,광업(05~08),B,광업
2,제조업(10~34),C,제조업
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업


In [31]:
plus["small"] = plus["Target"].str[1:4]
plus

,Dis,Target,BZ_PPOS_ITM_CTT,small
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업,
1,광업(05~08),B,광업,
2,제조업(10~34),C,제조업,
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업,
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업,
...,...,...,...,...
2016,가구 내 고용활동,T97000,가구 내 고용활동,970
2017,자가 소비를 위한 가사 생산 활동,T98100,자가 소비를 위한 가사 생산 활동,981
2018,자가 소비를 위한 가사 서비스 활동,T98200,자가 소비를 위한 가사 서비스 활동,982
2019,주한 외국 공관,U99001,주한 외국 공관,990


In [32]:
# 추가내용 파일 - 해당 분류에 맞게 삭제 
for i in range(len(plus)):
    if len(plus['small'][i])<3:
        plus=plus.drop(index=i)

print(len(plus))

1923


In [33]:
plus['small'].unique()

array(['011', '012', '013', '014', '015', '020', '031', '032', '051',
       '052', '061', '062', '071', '072', '080', '101', '102', '103',
       '104', '105', '106', '107', '108', '111', '112', '120', '131',
       '132', '133', '134', '139', '141', '142', '143', '144', '151',
       '152', '161', '162', '163', '171', '172', '179', '181', '182',
       '191', '192', '201', '202', '203', '204', '205', '211', '212',
       '213', '221', '222', '231', '232', '233', '239', '241', '242',
       '243', '251', '252', '259', '261', '262', '263', '264', '265',
       '266', '271', '272', '273', '274', '281', '282', '283', '284',
       '285', '289', '291', '292', '301', '302', '303', '304', '311',
       '312', '313', '319', '320', '331', '332', '333', '334', '339',
       '340', '351', '352', '353', '360', '370', '381', '382', '383',
       '390', '411', '412', '421', '422', '423', '424', '425', '426',
       '451', '452', '453', '461', '462', '463', '464', '465', '466',
       '467', '468',

In [34]:
data_small = plus.loc[:,['BZ_PPOS_ITM_CTT', 'small']]
data_small.tail()

,BZ_PPOS_ITM_CTT,small
2016,가구 내 고용활동,970
2017,자가 소비를 위한 가사 생산 활동,981
2018,자가 소비를 위한 가사 서비스 활동,982
2019,주한 외국 공관,990
2020,기타 국제 및 외국기관,990


In [35]:
ls = []
for i in range(len(train)):
    if len(train['KSIC10_BZC_CD'][i])<4:
        ls.append(i)
#         train=train.drop(index=i)
train.drop(index=ls,inplace=True)
print(len(train))

1121856


In [36]:
train['small']=train['KSIC10_BZC_CD'].str[1:4]
train.head()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,small
0,7695516.0,농산물 판매업 전자상거래업 통신판매업농임산물 건강기능식품 건강보조식품 일반식품...,G46209,G46,G,462
1,8177802.0,건축공사업 토목공사업 토공사업 토목건축공사업 철근콘크리트공사업 실내인테리...,F41229,F41,F,412
2,8233352.0,음식점업 경양식 퓨전 한식 중식 일식 식품 제조 가공 유통 판매 도소매업 제과...,I56111,I56,I,561
3,991833.0,변호사의 직무에 속하는 업무 공증인의 직무에 속하는 업무 변호사법 제조에 속하...,M71101,M71,M,711
4,7305137.0,일반폐기물처리업 재활용사업 정화조 청소업 위 각항에 부대되는 사업일체,E38210,E38,E,382


In [37]:
trn = pd.concat([train, data_small], ignore_index = True)
trn.tail()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,small
1123774,NaN,가구 내 고용활동,NaN,NaN,NaN,970
1123775,NaN,자가 소비를 위한 가사 생산 활동,NaN,NaN,NaN,981
1123776,NaN,자가 소비를 위한 가사 서비스 활동,NaN,NaN,NaN,982
1123777,NaN,주한 외국 공관,NaN,NaN,NaN,990
1123778,NaN,기타 국제 및 외국기관,NaN,NaN,NaN,990


In [38]:
trn = trn.loc[:,['BZ_PPOS_ITM_CTT', 'small']]
trn.tail()

,BZ_PPOS_ITM_CTT,small
1123774,가구 내 고용활동,970
1123775,자가 소비를 위한 가사 생산 활동,981
1123776,자가 소비를 위한 가사 서비스 활동,982
1123777,주한 외국 공관,990
1123778,기타 국제 및 외국기관,990


In [39]:
trn =trn.dropna(axis=0)
len(trn)

1123779

In [40]:
trn_shuffle = trn.sample(frac=1).reset_index(drop=True)
trn_shuffle.tail()

,BZ_PPOS_ITM_CTT,small
1123774,소프트웨어 개발 및 공급업 이러닝 교육학원업 및 홍보컨설팅업 온라인 광고업 ...,582
1123775,제조업 무역업 전기 통신업 도소매업 서비스업 인터넷 전자상거래업 소프트웨어 자문 개...,281
1123776,컴퓨터 및 주변장치 소프트웨어 도매업 컴퓨터 및 주변장치 소프트웨어 소매업 ...,465
1123777,구역화물자동차 운송사업 특수화물 자동차 운송사업 화물자동차 운송 알선사업 보...,493
1123778,인쇄제조업 무역업 축산업 기타 부대되는 사업의 일체,181


In [41]:
trn_shuffle['small'].unique()

array(['204', '473', '472', '251', '715', '423', '949', '478', '465',
       '412', '303', '222', '259', '262', '264', '492', '479', '161',
       '681', '461', '529', '713', '421', '464', '702', '239', '911',
       '493', '463', '411', '857', '011', '232', '320', '729', '202',
       '172', '422', '953', '721', '639', '752', '952', '292', '339',
       '739', '582', '741', '742', '352', '012', '291', '263', '631',
       '856', '302', '912', '162', '319', '468', '682', '241', '462',
       '642', '661', '941', '612', '311', '141', '759', '649', '467',
       '102', '424', '591', '351', '662', '561', '969', '107', '751',
       '871', '143', '452', '233', '071', '112', '620', '581', '272',
       '020', '340', '551', '872', '103', '285', '289', '032', '494',
       '732', '281', '466', '383', '108', '475', '381', '476', '101',
       '284', '192', '261', '181', '331', '014', '477', '592', '961',
       '211', '201', '762', '451', '471', '013', '701', '763', '474',
       '139', '370',

In [42]:
trn_shuffle.to_csv('preprocessing_data/train_data_small.csv', index = False)

In [ ]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(trn_shuffle['small'])
trn_shuffle['small_idx'] = encoder.transform(trn_shuffle['small'])
trn_shuffle.head()

In [ ]:
# 라벨링된 것 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
trn_shuffle = trn_shuffle.loc[:,['BZ_PPOS_ITM_CTT', 'small_idx']]
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('train_small.txt', sep='\t', index=True)

### 세세분류

In [49]:
train = pd.read_csv('train.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str})
plus = pd.read_csv('Augmentation_data.csv')

In [50]:
import re
plus["BZ_PPOS_ITM_CTT"] = plus["Dis"].str.replace('[^\w\s]', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업0103
1,광업(05~08),B,광업0508
2,제조업(10~34),C,제조업1034
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업35
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업3639


In [51]:
plus["BZ_PPOS_ITM_CTT"] = plus["BZ_PPOS_ITM_CTT"].str.replace('[0-9]+', '', regex=True)
plus.head()

,Dis,Target,BZ_PPOS_ITM_CTT
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업
1,광업(05~08),B,광업
2,제조업(10~34),C,제조업
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업


In [52]:
plus["sese"] = plus["Target"].str[1:]
plus

,Dis,Target,BZ_PPOS_ITM_CTT,sese
0,"농업, 임업 및 어업(01~03)",A,농업 임업 및 어업,
1,광업(05~08),B,광업,
2,제조업(10~34),C,제조업,
3,"전기, 가스, 증기 및 공기 조절 공급업(35)",D,전기 가스 증기 및 공기 조절 공급업,
4,"수도, 하수 및 폐기물 처리, 원료 재생업(36~39)",E,수도 하수 및 폐기물 처리 원료 재생업,
...,...,...,...,...
2016,가구 내 고용활동,T97000,가구 내 고용활동,97000
2017,자가 소비를 위한 가사 생산 활동,T98100,자가 소비를 위한 가사 생산 활동,98100
2018,자가 소비를 위한 가사 서비스 활동,T98200,자가 소비를 위한 가사 서비스 활동,98200
2019,주한 외국 공관,U99001,주한 외국 공관,99001


In [53]:
# 추가내용 파일 - 해당 분류에 맞게 삭제 
ls = []
for i in range(len(plus)):
    if len(plus['sese'][i])<5:
        ls.append(i)
        plus=plus.drop(index=i)

print(len(plus))

1196


In [54]:
plus['sese'].unique()

array(['01110', '01121', '01122', ..., '98200', '99001', '99009'],
      dtype=object)

In [55]:
data_sese = plus.loc[:,['BZ_PPOS_ITM_CTT', 'sese']]
data_sese.tail()

,BZ_PPOS_ITM_CTT,sese
2016,가구 내 고용활동,97000
2017,자가 소비를 위한 가사 생산 활동,98100
2018,자가 소비를 위한 가사 서비스 활동,98200
2019,주한 외국 공관,99001
2020,기타 국제 및 외국기관,99009


In [56]:
ls = []
for i in range(len(train)):
    if len(train['KSIC10_BZC_CD'][i])<6:
#         train=train.drop(index=i)
        ls.append(i)
train.drop(index=ls,inplace=True)
print(len(train))

1081146


In [57]:
train['sese']=train['KSIC10_BZC_CD'].str[1:6]
train.head()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,sese
0,7695516.0,농산물 판매업 전자상거래업 통신판매업농임산물 건강기능식품 건강보조식품 일반식품...,G46209,G46,G,46209
1,8177802.0,건축공사업 토목공사업 토공사업 토목건축공사업 철근콘크리트공사업 실내인테리...,F41229,F41,F,41229
2,8233352.0,음식점업 경양식 퓨전 한식 중식 일식 식품 제조 가공 유통 판매 도소매업 제과...,I56111,I56,I,56111
3,991833.0,변호사의 직무에 속하는 업무 공증인의 직무에 속하는 업무 변호사법 제조에 속하...,M71101,M71,M,71101
4,7305137.0,일반폐기물처리업 재활용사업 정화조 청소업 위 각항에 부대되는 사업일체,E38210,E38,E,38210


In [58]:
trn = pd.concat([train, data_sese], ignore_index = True)
trn.tail()

,KEDCD,BZ_PPOS_ITM_CTT,KSIC10_BZC_CD,mid,Big,sese
1082337,NaN,가구 내 고용활동,NaN,NaN,NaN,97000
1082338,NaN,자가 소비를 위한 가사 생산 활동,NaN,NaN,NaN,98100
1082339,NaN,자가 소비를 위한 가사 서비스 활동,NaN,NaN,NaN,98200
1082340,NaN,주한 외국 공관,NaN,NaN,NaN,99001
1082341,NaN,기타 국제 및 외국기관,NaN,NaN,NaN,99009


In [59]:
trn = trn.loc[:,['BZ_PPOS_ITM_CTT', 'sese']]
trn.tail()

,BZ_PPOS_ITM_CTT,sese
1082337,가구 내 고용활동,97000
1082338,자가 소비를 위한 가사 생산 활동,98100
1082339,자가 소비를 위한 가사 서비스 활동,98200
1082340,주한 외국 공관,99001
1082341,기타 국제 및 외국기관,99009


In [60]:
trn =trn.dropna(axis=0)
len(trn)

1082342

In [61]:
trn_shuffle = trn.sample(frac=1).reset_index(drop=True)
trn_shuffle.tail()

,BZ_PPOS_ITM_CTT,sese
1082337,적 농업경영을 통하여 생산성을 높이고 농산물의 출하가공수출등을 통하여 조합원의 소 ...,01159
1082338,유류 보관판매 및 도소매업 유류 수송업 주유소 운영업 섬유 유제 제조 및 도...,46712
1082339,철구조물 제작업 부동산임대업 토목공사업 건설장비 판매업 위 각호에 부대되는...,42600
1082340,피에프사업 부동산개발 공급 매매 임대업 기타관련서비스업 주택법에 의한 주택건설사업 ...,68129
1082341,조합법인은 협업적 농업경영을 통하여 생산성을 높이고 농산물의 공동출하 및 가공 수출...,01159


In [62]:
trn_shuffle.to_csv('preprocessing_data/train_data_sese.csv', index = False)

In [ ]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(trn_shuffle['sese'])
trn_shuffle['sese_idx'] = encoder.transform(trn_shuffle['sese'])
trn_shuffle.head()

In [ ]:
# 라벨링된 것 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
trn_shuffle = trn_shuffle.loc[:,['BZ_PPOS_ITM_CTT', 'sese_idx']]
trn_shuffle.head()

In [ ]:
trn_shuffle.to_csv('train_sese.txt', sep='\t', index=True)

## Preprocessing for LSTM(세분류)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
from konlpy.tag import Mecab

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
mecab = Mecab()
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']
X_train = []

text = list(train['BZ_PPOS_ITM_CTT'])

for i in tqdm(range(len(text))):
    temp_X = []
    temp_X = mecab.nouns(text[i]) # 토큰화
    temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]
    X_train.append(temp_X)
X_test = []

text = list(test['BZ_PPOS_ITM_CTT'])

for i in tqdm(range(len(text))):
    temp_X = []
    temp_X = mecab.nouns(text[i]) # 토큰화
    temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
    # temp_X = [word for word in temp_X if len(word) > 1]
    X_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 11
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = total_cnt - rare_cnt + 1 # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size) # num_words = vocab_size
tokenizer.fit_on_texts(X_train)
token_X_train = tokenizer.texts_to_sequences(X_train)
token_X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
y_train = to_categorical(np.array(train['KSIC10_BZC_CD']))
drop_train = [index for index, sentence in enumerate(token_X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
del_X_train = np.delete(token_X_train, drop_train, axis=0)
del_y_train = np.delete(y_train, drop_train, axis=0)
print(len(del_X_train))
print(len(del_y_train))

In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 300
below_threshold_len(max_len, del_X_train)

In [ ]:
pad_X_train = pad_sequences(del_X_train, maxlen = max_len)
pad_X_test = pad_sequences(token_X_test, maxlen = max_len)

## Train & Test Bert

### 대분류

In [ ]:
!git clone https://github.com/yoosoobin/potato
!python bert_train.py --source train_big.txt --device 0 --save-weight-model final_big --classes 21 --batch 64

In [ ]:
!python bert_test.py --source bert_test.txt --weights final_big.pt --classes 21 --batch 32 --device 0 --save_csv_name big_test_pred

In [ ]:
big = pd.read_csv('big_test_pred.csv')

In [ ]:
import string
dic = {}
for k,v in zip([i for i in range(21)],string.ascii_uppercase):
    dic[k] = v

In [ ]:
submit = pd.DataFrame(columns = ['KEDCD','big'])

In [ ]:
submit['KEDCD'] = test_df['KEDCD'].astype(str)
submit['pred_big'] = big['pred'].astype(int).map(dic)

### 중분류

In [ ]:
!python bert_train.py --source train_mid.txt --device 0 --save-weight-model final_mid --classes 77 --batch 64

In [ ]:
!python bert_test.py --source bert_test.txt --weights final_mid.pt --classes 77 --batch 32 --device 0 --save_csv_name mid_test_pred

In [ ]:
mid = pd.read_csv('mid_test_pred.csv')

In [ ]:
train_mid = pd.read_csv('preprocessing_data/train_data_mid.csv',dtype={'mid':str})

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_mid['mid'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))


In [ ]:
mid['label'] = mid['pred'].astype(int).map(mapping)

In [ ]:
submit['mid'] = mid['label']

### 소분류

In [ ]:
!python bert_train.py --source train_small.txt --device 0 --save-weight-model final_small --classes 232 --batch 64

In [ ]:
!python bert_test.py --source bert_test.txt --weights final_small.pt --classes 232 --batch 32 --device 0 --save_csv_name small_test_pred

In [ ]:
small = pd.read_csv('small_test_pred.csv')

In [ ]:
train_small = pd.read_csv('preprocessing_data/train_data_small.csv',dtype={'small':str})

In [ ]:
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() # encoding
encoder.fit(train_small['small'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_)) # decoding

In [ ]:
small['label'] = small['pred'].astype(int).map(mapping)  # decoding mapping

In [ ]:
submit['small'] = small['label']

### 세분류(LSTM)

In [ ]:
model4 = Sequential()
model4.add(Embedding(vocab_size, 32, input_length=max_len))
model4.add(Dropout(0.3))
model4.add(Conv1D(32, 5, activation='relu'))
model4.add(Conv1D(32, 5, activation='relu'))
model4.add(MaxPooling1D(pool_size=4))
model4.add(LSTM(32))
model4.add(BatchNormalization())
model4.add(Dense(491, activation='softmax'))
model4.summary()

In [ ]:
model4.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = .0005), metrics=['acc'])
callback_ear = [EarlyStopping(monitor = 'loss', mode = 'min', patience = 5, verbose = 1), ModelCheckpoint(filepath =  os.path.join(path, 'se_model_0520'), monitor = 'val_acc', save_best_only = True, mode = 'max', verbose = 1)]

history4 = model4.fit(pad_X_train, se_y_train, epochs=100, batch_size=150, validation_split=val_rate, verbose = 1,shuffle = True, callbacks= callback_ear)

In [ ]:
model4 = load_model( os.path.join(path, 'se_model_0520'))

In [ ]:
pred4 = model4.predict(pad_X_test)

In [ ]:
train_se=pd.read_csv('train_data_se.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str,'small':str,'se':str})

In [ ]:
ls = train_se['se'].unique().tolist()

In [ ]:
final_ls = []
for i in ls:
    if len(i)==4:
        final_ls.append(i)
final_ls = sorted(final_ls)

In [ ]:
pred_se_val = []
for i in pred_X_test:
    pred_idx = np.argmax(i)
    pred_se_val.append(final_ls[pred_idx])
submit['se'] = pred_se_val

### 세세분류

In [ ]:
!python bert_train.py --source train_sese.txt --device 0 --save-weight-model final_sese --classes 1196 --batch 64

In [ ]:
!python bert_test.py --source bert_test.txt --weights final_sese.pt --classes 1196 --batch 32 --device 0 --save_csv_name sese_test_pred

In [ ]:
train_sese = pd.read_csv('train_data_sese.csv')
uni = final_train['KSIC10_BZC_CD'].unique()
sese_list = []
for i in uni.astype(str):
    if len(i) == 6:
        sese_list.append(i)
real_sese_list = sorted(sese_list)
real_sese_list = list(map(lambda x: x[1:],real_sese_list))

In [ ]:
sese = pd.read_csv('sese_test_pred.csv')

In [ ]:
train_sese = pd.read_csv('preprocessing_data/train_data_sese.csv',dtype={'sese':str})

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_sese['sese'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))

In [ ]:
sese['label'] = sese['pred'].astype(int).map(mapping)

In [ ]:
submit['sese'] = sese['label']

## Final Predict

In [ ]:
final_mid = []
final_small = []
# final_se = []
for i in range(len(submit)):
    # 중분류
    ls = []
    mid = submit.iloc[i]['mid']
    small = submit.iloc[i]['small']
    se = submit.iloc[i]['se']
    sese = submit.iloc[i]['sese']
    ls.append(mid)
    ls.append(small[:-1])
    ls.append(se[:-2])
    ls.append(sese[:-3])
    counting = pd.Series(ls).value_counts()
    if counting.values[0] == 1:
        final_mid.append(mid)
    elif counting.values[0] == 2:
        if counting.values[0] == 2:
            final_mid.append(mid)
    else:
        final_mid.append(counting.index[0])
    # 소분류
    ls2 = []
    ls2.append(small[-1])
    ls2.append(se[-2])
    ls2.append(sese[-3])
    counting2 = pd.Series(ls2).value_counts()
    if counting2.values[0] == 1:
        final_small.append(small[-1])
    else:
        final_small.append(counting2.index[0])

In [ ]:
submit['final_mid'] = final_mid
submit['final_small'] = final_small
submit['pred'] = submit['big']+submit['final_mid'] + submit['final_small'].str[-1] + submit['se'].str[-1] + submit['sese'].str[-1]

In [ ]:
submit.drop(['se','big','mid','small','sese','final_mid','final_small'],axis=1,inplace=True)

In [ ]:
submit = submit[submit['KEDCD'] != '6525199']

In [ ]:
result = pd.read_excel('[공모전]정답제출양식_(조)_제출일자.xlsx')
result = pd.merge(result,submit,on='KEDCD')
result.drop('모델이 예측한 업종코드',axis=1,inplace=True)
result.columns = ['KEDCD','모델이 예측한 업종코드']
result.to_excel('[공모전]정답제출양식_(애쓰는감자)_20210523.xlsx',index=False)